### Word Embedding Manual with Creating token_type_ids

In [ ]:
from transformers import BertTokenizer
import torch

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input dengan lebih dari dua segmen
segments = [
    "What is photosynthesis?",  # Segmen 1
    "Photosynthesis is the process by which plants convert sunlight into energy.",  # Segmen 2
    "It occurs in the chloroplasts of plant cells."  # Segmen 3
]

# Encode masing-masing segmen
encoded_segments = [tokenizer.encode(seg, add_special_tokens=False) for seg in segments]

# Gabungkan segmen dengan [SEP] di antaranya
input_ids = [tokenizer.cls_token_id]  # [CLS]
token_type_ids = []  # Untuk menyimpan ID tipe token
current_segment_id = 0

for segment in encoded_segments:
    input_ids.extend(segment + [tokenizer.sep_token_id])  # Tambahkan segmen dan [SEP]
    token_type_ids.extend([current_segment_id] * (len(segment) + 1))  # Token Type IDs
    current_segment_id += 1  # Pindah ke segmen berikutnya

# Padding untuk mencapai panjang maksimum
max_length = 50
attention_mask = [1] * len(input_ids)  # Mask untuk token yang relevan

# Tambahkan padding jika diperlukan
while len(input_ids) < max_length:
    input_ids.append(0)  # Token PAD
    attention_mask.append(0)
    token_type_ids.append(0)  # Token Type ID untuk padding

# Pastikan panjangnya sesuai
input_ids = input_ids[:max_length]
attention_mask = attention_mask[:max_length]
token_type_ids = token_type_ids[:max_length]

# Konversi ke tensor PyTorch
input_ids = torch.tensor([input_ids])
attention_mask = torch.tensor([attention_mask])
token_type_ids = torch.tensor([token_type_ids])

# Output
print("Input IDs:", input_ids)
print("Attention Mask:", attention_mask)
print("Token Type IDs:", token_type_ids)


# Adding custom embedding to handle more than 2 segment in BERT model varian

In [ ]:
import torch
import torch.nn as nn
from transformers import AlbertModel

class CustomAlbertModel(nn.Module):
    def __init__(self, model_name='indobenchmark/indobert-lite-base-p2', num_token_types=3):
        super().__init__()
        # Load ALBERT model
        self.albert = AlbertModel.from_pretrained(model_name)
        
        # Replace token_type_embeddings to support more token types
        self.albert.embeddings.token_type_embeddings = nn.Embedding(
            num_embeddings=num_token_types, 
            embedding_dim=self.albert.config.hidden_size
        )
    
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        # Pass through modified ALBERT model
        return self.albert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids
        )

# Create model instance
model = CustomAlbertModel(num_token_types=3)

# Test model
input_ids = torch.randint(0, 30000, (4, 512))  # Example input
attention_mask = torch.ones(4, 512)  # Example mask
token_type_ids = torch.randint(0, 3, (4, 512))  # Example token types (0, 1, 2)

output = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
print(output.last_hidden_state.shape)  # Should be (batch_size, sequence_length, hidden_size)
